In [1]:
import pandas as pd
import cv2
data=pd.read_csv('/kaggle/input/affecnet-dataset/Affectnet Dataset/labels.csv')
data.sample(5)

,Unnamed: 0,pth,label,relFCs
2832,2832,anger/image0029735.jpg,anger,0.896594
24819,24819,surprise/image0000139.jpg,fear,0.754212
15773,15773,happy/ffhq_664.png,happy,0.812568
26085,26085,surprise/image0023632.jpg,neutral,0.617558
18977,18977,neutral/ffhq_4105.png,disgust,0.813096


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28175 entries, 0 to 28174
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  28175 non-null  int64  
 1   pth         28175 non-null  object 
 2   label       28175 non-null  object 
 3   relFCs      28175 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 880.6+ KB


In [3]:
X = data['pth'].values
y = data['label'].values

In [4]:
import cv2
from keras.utils import to_categorical

def crop_face(image_path):
    img = cv2.imread(image_path)
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    clf = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    #img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    faces = clf.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5, minSize=(30,30),flags=cv2.CASCADE_SCALE_IMAGE)
    
    if len(faces) == 0:
        return None

    (x, y, w, h) = faces[0]
    
    # Crop the detected face region from the image
    cropped_face = img[y:y+h, x:x+w]
    #print(len(cropped_face))
    return cropped_face


2024-04-26 07:47:26.666704: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-26 07:47:26.666821: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-26 07:47:26.770701: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
print(data.iloc[19]['pth'])

anger/image0000399.jpg


In [6]:
#mg = cv2.imread("/kaggle/input/affecnet-dataset/Affectnet Dataset/"+data.iloc[19]['pth'])
#temp = crop_face("/kaggle/input/affecnet-dataset/Affectnet Dataset/"+data.iloc[90]['pth'])
#temp.shape[0]

Finding Max size of image

In [7]:
'''MAX_SIZE=0
for i in range(0,28175):
    data_dir = "/kaggle/input/affecnet-dataset/Affectnet Dataset/"+data.iloc[i]['pth']
    cropped_face = crop_face(data_dir)
    if cropped_face is not None:
        temp=cropped_face.shape[0]
        if  temp> MAX_SIZE:
            MAX_SIZE=temp
            print(MAX_SIZE)'''

'MAX_SIZE=0\nfor i in range(0,28175):\n    data_dir = "/kaggle/input/affecnet-dataset/Affectnet Dataset/"+data.iloc[i][\'pth\']\n    cropped_face = crop_face(data_dir)\n    if cropped_face is not None:\n        temp=cropped_face.shape[0]\n        if  temp> MAX_SIZE:\n            MAX_SIZE=temp\n            print(MAX_SIZE)'

In [8]:
#img = cv2.imread("/kaggle/input/affecnet-dataset/Affectnet Dataset/"+data.iloc[19]['pth'])
#import matplotlib.pyplot as plt
#plt.imshow(img)

In [9]:
'''df=pd.DataFrame(X_preprocessed)
df.to_csv('X_preprocessed_1.csv',index=False,header=False)
df.to_csv('Y_preprocessed_1.csv',index=False,header=False)'''

"df=pd.DataFrame(X_preprocessed)\ndf.to_csv('X_preprocessed_1.csv',index=False,header=False)\ndf.to_csv('Y_preprocessed_1.csv',index=False,header=False)"

In [10]:
'''import numpy as np

X_preprocessed_array = np.array(X_preprocessed)
Y_preprocessed_array = np.array(Y_preprocessed)'''

'import numpy as np\n\nX_preprocessed_array = np.array(X_preprocessed)\nY_preprocessed_array = np.array(Y_preprocessed)'

In [11]:
lost=0
X_preprocessed=[]
Y_preprocessed=[]

for i in range(0,28175):
#for i in range(0,4695):
    data_dir = "/kaggle/input/affecnet-dataset/Affectnet Dataset/"+data.iloc[i]['pth']
    
    #img = cv2.imread(data_dir)
    cropped_face = crop_face(data_dir)
    if cropped_face is not None:
        resized_face = cv2.resize(cropped_face, (88, 88))
        normalised=resized_face/255.0
        X_preprocessed.append(normalised)
        Y_preprocessed.append(y[i])
        #print(f"X-->{len(X_preprocessed)} Y-->{len(Y_preprocessed)}")

    '''else:
        lost+=1
        print(f"lost-->{lost}<--")'''

In [12]:
len(Y_preprocessed)

26509

In [13]:
import numpy as np

X_preprocessed = np.array(X_preprocessed)
Y_preprocessed = np.array(Y_preprocessed)

In [14]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y_preprocessed = to_categorical(le.fit_transform(Y_preprocessed))

In [15]:
Y_preprocessed[0]

array([0., 0., 0., 0., 0., 0., 0., 1.])

In [16]:
le.inverse_transform([0, 1, 2, 3, 4, 5, 6, 7])

array(['anger', 'contempt', 'disgust', 'fear', 'happy', 'neutral', 'sad',
       'surprise'], dtype='<U8')

In [17]:
from sklearn.model_selection import train_test_split

# Split the preprocessed data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, Y_preprocessed, test_size=0.2, random_state=42, stratify=Y_preprocessed)


In [18]:
type(y_test)

numpy.ndarray

In [19]:
X_preprocessed =[]
Y_preprocessed = []

In [20]:
'''import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(88, 88, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(8, activation='softmax')  # Assuming 8 emotion classes
])'''


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [52]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Data augmentation
'''data_augmentation = tf.keras.Sequential([
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomFlip("horizontal"),
])'''

# Define the model
model = models.Sequential([
    #data_augmentation,
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(88, 88, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  # Dropout layer for regularization
    layers.Dense(8, activation='softmax')  # Assuming 8 emotion classes
])

# Compile the model

# Train the model
#history = model.fit(X_train, y_train, epochs=60, batch_size=64, validation_split=0.3)


In [55]:
#model.compile(optimizer='adam',
#              loss='categorical_crossentropy',  # Use categorical_crossentropy for one-hot encoded labels
#              metrics=['accuracy'])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [56]:
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 86, 86, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 43, 43, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 41, 41, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 20, 20, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 18, 18, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 9, 9, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 10368)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │     1,327,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 8)              │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,421,512 (5.42 MB)

 Trainable params: 1,421,512 (5.42 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
#!pip install visualkeras
'''import visualkeras
visualkeras.layered_view(model).show() # display using your system viewer
visualkeras.layered_view(model, to_file='Neural_Network_Visualize.png') # write to disk
visualkeras.layered_view(model, to_file='Neural_Network_Visualize.png').show() # write and show'''

"import visualkeras\nvisualkeras.layered_view(model).show() # display using your system viewer\nvisualkeras.layered_view(model, to_file='Neural_Network_Visualize.png') # write to disk\nvisualkeras.layered_view(model, to_file='Neural_Network_Visualize.png').show() # write and show"

In [57]:
# Train the model
model.fit(X_train, y_train, epochs=60, batch_size=64, validation_split=0.3)

Epoch 1/60
 13/232 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.1188 - loss: 2.1338

W0000 00:00:1714122851.746631      95 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.1857 - loss: 2.0371

W0000 00:00:1714122857.731846      97 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1714122859.690083      95 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


232/232 ━━━━━━━━━━━━━━━━━━━━ 14s 39ms/step - accuracy: 0.1860 - loss: 2.0367 - val_accuracy: 0.3720 - val_loss: 1.6963
Epoch 2/60
232/232 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.3714 - loss: 1.6938 - val_accuracy: 0.4492 - val_loss: 1.5014
Epoch 3/60
232/232 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.4410 - loss: 1.5127 - val_accuracy: 0.5232 - val_loss: 1.3480
Epoch 4/60
232/232 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.4756 - loss: 1.3877 - val_accuracy: 0.5285 - val_loss: 1.3036
Epoch 5/60
232/232 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.5074 - loss: 1.3228 - val_accuracy: 0.5098 - val_loss: 1.3375
Epoch 6/60
232/232 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.5215 - loss: 1.2666 - val_accuracy: 0.5493 - val_loss: 1.2119
Epoch 7/60
232/232 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.5491 - loss: 1.1918 - val_accuracy: 0.5681 - val_loss: 1.1741
Epoch 8/60
232/232 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.5692 - loss: 1.1450 - val_accuracy: 0.5

In [59]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")

166/166 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5763 - loss: 2.7958
Test Accuracy: 0.5809128880500793


In [26]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")

166/166 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4967 - loss: 5.8283
Test Accuracy: 0.5047152042388916


In [27]:
type(X_train[0])

numpy.ndarray

In [60]:
#img = cv2.imread("/kaggle/input/affecnet-dataset/Affectnet Dataset/fear/image0000284.jpg")
crop_img=crop_face("/kaggle/input/affecnet-dataset/Affectnet Dataset/fear/image0000284.jpg")
resized_face = cv2.resize(crop_img, (88, 88))
normalised=resized_face/255.0
normalised=np.array(normalised)

type(normalised)

numpy.ndarray

In [66]:
normalised.shape

(88, 88, 3)

In [67]:
model.predict(normalised)

ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(32, 88, 3), dtype=float32). Expected shape (None, 88, 88, 3), but input has incompatible shape (32, 88, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(32, 88, 3), dtype=float32)
  • training=False
  • mask=None

In [69]:
import pickle
with open('Model_pickle','wb') as f:
    pickle.dump(model,f)

AttributeError: Can't pickle local object 'Layer._initializer_tracker.<locals>.<lambda>'

In [68]:
model.save('/kaggle/working/experimental')
model.save_weights('/kaggle/working/Scaled_model.h5')

ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `tf.saved_model.save()` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=/kaggle/working/experimental.

In [ ]:
'''df=pd.DataFrame(X_preprocessed)
df.to_csv('X_preprocessed_1.csv',index=False,header=False)
df=pd.DataFrame(Y_preprocessed)
df.to_csv('Y_preprocessed_1.csv',index=False,header=False)'''

In [ ]:
'''X_preprocessed_array = np.concatenate((X_preprocessed_array, np.array(X_preprocessed)))
Y_preprocessed_array = np.concatenate((Y_preprocessed_array, np.array(Y_preprocessed)))'''